In [16]:
#Import Libraries
import sys
import os
import pandas as pd
from datetime import datetime

In [17]:
# Add the '../scripts' directory to the Python module search path.
# This allows us to import modules from that directory, 
# facilitating better organization of our code and access to shared scripts.
sys.path.append(os.path.abspath('../scripts'))

In [18]:
# Import the modules
import data_processing as dp
import model as md


In [19]:

df=pd.read_csv('../data/cleanedDataset.csv')
df.head(2)

C:\Users\asmam\AppData\Local\Temp\ipykernel_13940\4078464814.py:1: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('../data/cleanedDataset.csv')


,UnderwrittenCoverID,PolicyID,PostalCode,mmcode,RegistrationYear,Cylinders,cubiccapacity,kilowatts,NumberOfDoors,CustomValueEstimate,...,CrossBorder,TermFrequency,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType
0,120087,10173,1619,54057250.0,2004,4.0,1870.0,75.0,5.0,52700.0,...,Unknown,Monthly,No excess,Signage and Vehicle Wraps,Signage and Vehicle Wraps,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant
1,120087,10173,1619,54057250.0,2004,4.0,1870.0,75.0,5.0,52700.0,...,Unknown,Monthly,No excess,Signage and Vehicle Wraps,Signage and Vehicle Wraps,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant


In [20]:
df['TransactionMonth']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
882929   NaN
882930   NaN
882931   NaN
882932   NaN
882933   NaN
Name: TransactionMonth, Length: 882934, dtype: float64

In [21]:

# for col in df.columns:
#     unique_values = df[col].nunique()
#     if unique_values<=10:
#         unique_values_lists = df[col].unique()
#         print(f"Unique values in {col}-->{unique_values}-->: {unique_values_lists}\n")


In [22]:
# Get categorical columns
categorical_columns = df.select_dtypes(include='object').columns
dp.column_detail(df,categorical_columns)

,Column,DataType,NumUniqueValues
0,Citizenship,object,4
1,LegalType,object,6
2,Title,object,5
3,Language,object,1
4,Bank,object,12
5,AccountType,object,4
6,MaritalStatus,object,4
7,Gender,object,4
8,Country,object,1
9,Province,object,9


. Language, Country, ItemType, StatutoryClass, and StatutoryRiskType don't provide variability and drop them.

In [23]:
columns_to_drop = ['Language', 'Country', 'ItemType', 'StatutoryClass', 'StatutoryRiskType']
df=dp.drop_columns(df,columns_to_drop)

In [24]:
# Convert boolean 'IsVATRegistered' to integer
df['IsVATRegistered'] = df['IsVATRegistered'].astype(int)

In [25]:
dp.distribution(df)

Gender Distribution:
Gender
Not specified    828251
Male              39309
Unknown            9413
Female             5961
Name: count, dtype: int64
Title Distribution:
 Title
Mr      822631
Mrs      41478
Ms       11598
Miss      6426
Dr         801
Name: count, dtype: int64
Marital Status Distribution:
 MaritalStatus
Not specified    870209
Unknown            8133
Single             4123
Married             469
Name: count, dtype: int64


#### A significant portion of the Gender values (828251) is unspecified, and can be filled using the information from the Title column where available.

#### Mr → Male

#### Mrs, Miss, Ms → Female

#### Dr → Leave as Not specified (since it’s ambiguous)

In [26]:
# Map Titles to Gender
title_to_gender_map = {
    'Mr': 'Male',
    'Mrs': 'Female',
    'Ms': 'Female',
    'Miss': 'Female',
    'Dr': None  # Leave as None since it could be either gender
}
dp.title_to_gender_map(df,title_to_gender_map)

Gender
Male      822631
Female     59502
Name: count, dtype: int64


In [28]:
df.columns

Index(['UnderwrittenCoverID', 'PolicyID', 'PostalCode', 'mmcode',
       'RegistrationYear', 'Cylinders', 'cubiccapacity', 'kilowatts',
       'NumberOfDoors', 'CustomValueEstimate', 'SumInsured',
       'CalculatedPremiumPerTerm', 'TotalPremium', 'TotalClaims',
       'TransactionMonth', 'IsVATRegistered', 'Citizenship', 'LegalType',
       'Title', 'Bank', 'AccountType', 'Gender', 'Province', 'MainCrestaZone',
       'SubCrestaZone', 'VehicleType', 'make', 'Model', 'bodytype',
       'VehicleIntroDate', 'AlarmImmobiliser', 'TrackingDevice',
       'CapitalOutstanding', 'NewVehicle', 'WrittenOff', 'Rebuilt',
       'Converted', 'CrossBorder', 'TermFrequency', 'ExcessSelected',
       'CoverCategory', 'CoverType', 'CoverGroup', 'Section', 'Product'],
      dtype='object')

In [30]:
# Drop the 'MaritalStatus' column from the DataFrame which have high proportion of unspecified values
df = df.drop(columns=['MaritalStatus'])
# Drop the specified columns from the DataFrame
df = df.drop(columns=['UnderwrittenCoverID', 'UnderwrittenCoverID', 'PolicyID'])

In [34]:
# Check which columns have non-numeric values
non_numeric_columns = df.select_dtypes(include=['object']).columns
print("Columns with non-numeric values:", non_numeric_columns)

# Inspect rows with problematic values
for col in non_numeric_columns:
    print(f"Values in {col}: {df[col].unique()}")


Columns with non-numeric values: Index(['Citizenship', 'LegalType', 'Title', 'Bank', 'AccountType', 'Gender',
       'Province', 'MainCrestaZone', 'SubCrestaZone', 'VehicleType', 'make',
       'Model', 'bodytype', 'AlarmImmobiliser', 'TrackingDevice',
       'CapitalOutstanding', 'NewVehicle', 'WrittenOff', 'Rebuilt',
       'Converted', 'CrossBorder', 'TermFrequency', 'ExcessSelected',
       'CoverCategory', 'CoverType', 'CoverGroup', 'Section', 'Product'],
      dtype='object')
Values in Citizenship: ['  ' 'AF' 'ZA' 'ZW']
Values in LegalType: ['Close Corporation' 'Individual' 'Partnership' 'Private company'
 'Public company' 'Sole proprieter']
Values in Title: ['Mr' 'Dr' 'Miss' 'Mrs' 'Ms']
Values in Bank: ['First National Bank' 'Standard Bank' 'Unknown' 'ABSA Bank'
 'Capitec Bank' 'Nedbank' 'FirstRand Bank' 'Investec Bank' 'Ithala Bank'
 'Old Mutual' 'Mercantile Lisbon Bank' 'RMB Private Bank']
Values in AccountType: ['Current account' 'Savings account' 'Unknown' 'Transmission acco

In [32]:


# Load and clean data
filepath = '../data/cleanedDataset.csv'
data = dp.load_and_clean_data(filepath)

# Encode and scale the data
# Columns to be encoded
columns_label = ['Province', 'PostalCode', 'Gender']
columns_onehot = ['Province', 'PostalCode', 'Gender']

df_label = dp.encoder('labelEncoder', data, columns_label, columns_onehot)

columns_scaler = ['TotalPremium']
df_scaled = dp.scaler('minMaxScaler', df_label, columns_scaler)

# Split the data
X = df_scaled.drop('TotalPremium', axis=1)
y = df_scaled['TotalPremium']
X_train, X_test, y_train, y_test = md.split_data(X, y)

# Train models
lr_model, dt_model, rfr_model, xgb_model = md.train_models(X_train, y_train)

# Evaluate models
models = ['Linear Regression', 'Decision Tree', 'Random Forest', 'XGBoost']
mae_scores, mse_scores, r2_scores = [], [], []
for model in [lr_model, dt_model, rfr_model, xgb_model]:
    mae, mse, r2, _ = md.evaluate_model(model, X_test, y_test)
    mae_scores.append(mae)
    mse_scores.append(mse)
    r2_scores.append(r2)

# Plot metrics
md.plot_metrics(models, mae_scores, mse_scores, r2_scores)


d:\Kifya\Week_03\scripts\data_processing.py:8: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filepath)


ValueError: could not convert string to float: '  '